In [70]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [73]:
%env twitter_api_key

'pFlQlVe3RxDDloIFv3Wq3lv2I'

In [74]:
# criar um dataset com dados de usuários do twitter + seus x tweets.

In [75]:
# packages imports here
import tweepy
import pandas as pd

In [76]:
# Loading variables for tweepy

twitter = {}
twitter['TWITTER_API_KEY'] = %env twitter_api_key 
twitter['TWITTER_API_SECRET'] = %env twitter_api_secret
twitter['TWITTER_API_TOKEN'] = %env twitter_access_token
twitter['TWITTER_API_TOKEN_SECRET'] = %env twitter_access_token_secret


#for key in twitter:
#    print(key, '->', twitter[key])

# Authenticate with Twitter API

twitter['TWITTER_API_TOKEN_SECRET']
auth = tweepy.OAuthHandler(twitter['TWITTER_API_KEY'], twitter['TWITTER_API_SECRET'])

# se não autenticar com o access token algumas funcionalidades da api 
# não ficam disponíveis. tipo verificar rate limits.
auth.set_access_token(twitter['TWITTER_API_TOKEN'], twitter['TWITTER_API_TOKEN_SECRET'])

# o objeto api é utilizado para realizar toda comunicação com a API do twitter.
api = tweepy.API(auth)

In [77]:
# recuperando dados de um twitter user
def getDataFromUserByTwiiterHandle(handle):
    try:
        user = api.get_user(screen_name = handle)
        return user
    except tweepy.TweepError as e:
#         print("Tweepy Error retrieving user: {}".format(e))
        return e
#         return {'reason' : e.reason, 'codes' : e.api_code, 'response' : e.response, 'args' : e.args}

In [78]:
# recuperando dados de um twitter user
def getUserTimeline(uid, num_tweets = 100):
    try:       
        timeline = tweepy.Cursor(api.user_timeline, user_id = uid, count=num_tweets).items(num_tweets)
        return timeline
    except tweepy.TweepError as e:
#         print("Tweepy Error timeline: {}".format(e))
        return {'reason' : e.reason, 'codes' : e.api_code, 'response' : e.response, 'args' : e.args}

In [79]:
# create user dataframe by list of users retrieved by tweepy
def createDataFrameFromUser(user, dfUsers, error=False, handle=None):
#     dfUsers = pd.DataFrame();
#     for user in users:
    if error == False:
        u = {
            'twitter_id': user.id,
            'name': user.name,
            'handle' : user.screen_name,
            'twitter_is_protected': user.protected,
            'description': user.description,
            'followers_count': user.followers_count,
            'friends_count': user.friends_count,
            'location': user.location,
            'created_at': user.created_at,
            'verified': user.verified,
            'lang': user.lang,
            'default_profile': user.default_profile,
            'profile_image': user.profile_image_url,
            'withheld_in_countries' : user.withheld_in_countries
        }
        dfUsers = dfUsers.append(u, ignore_index=True)
    else:
        e = {'error': user, 'handle' : handle}
        dfUsers = dfUsers.append(e, ignore_index=True)
    return dfUsers

In [80]:
def createDataFrameFromTimeline(timeline, dfTimeline, error = False, handle=None):
    if error == False:
        for tweet in timeline:
            t = {
                'tweet_id' : tweet.id,
                'tweet_author': tweet.author.screen_name,
                'tweet_id_str' : tweet.id_str,
                'tweet_retweeted' : tweet.retweeted,
                'tweet_created_at': tweet.created_at,
                'tweet_source' : tweet.source,
                'tweet_author_id_str':tweet.author.id_str,
                'tweet_text':tweet.text,
                'tweet_contributors': tweet.contributors,
                'tweet_favorite_count':tweet.favorite_count,
                'tweet_favorited' : tweet.favorited,
                'tweet_geo' : tweet.geo,
                'tweet_is_retweet': tweet.is_quote_status,
                'tweet_lang' : tweet.lang,
                'tweet_place' : tweet.place,
                'tweet_hashtags' : list(map(lambda x:x['text'], tweet.entities['hashtags']? ''))
            }
            dfTimeline = dfTimeline.append(t, ignore_index=True)
    else:
        e = {'error': timeline, 'handle' : handle}
        dfTimeline = dfTimeline.append(e, ignore_index=True)
    return dfTimeline    

In [81]:
twitterUser = getDataFromUserByTwiiterHandle("blckjzzzzz")
print(type(twitterUser))

<class 'tweepy.error.TweepError'>


In [82]:
#handles = pd.read_csv("/Users/dc/Downloads/teste.csv", skiprows=[0], header=None,
df2 = pd.read_csv("/Users/dc/Downloads/teste.csv",
                      delimiter=",", encoding="latin")

df2.head()

len(df2)

98

In [83]:
# dataframes creation

columns = {
    'error'
}

dfUser = pd.DataFrame(columns=columns);
dfTimelines = pd.DataFrame(columns=columns);

#for handle in handles:
# handles = {'blckjxxxxxxzz', 'VieiraClever1', 'blckjzz'} # testing handles {do not exist, blocked timeline, real}

print(f' ############ {len(df2)} # PERFIS PARA ANALISAR')
for index, handle in df2.itertuples(): 
    print(f'{index+1}    #    Analisando o perfil:@ {handle}')
    twitterUser = getDataFromUserByTwiiterHandle(handle)
    # if its true, the twitter api has returned some error
    # False, should mean that the user is found and you can work with it.
#     print(type(twitterUser))
    if isinstance(twitterUser, tweepy.error.TweepError) == False: # means that the user exists because the Twitter API response
#         print(f'Encontrei o {handle}. Seu nome é: {twitterUser.name}')
        # When you have the user retrieved
        # The Expected flow should be written here
        dfUser = createDataFrameFromUser(twitterUser, dfUser)
        if twitterUser.protected == False: #means that the timeline is open and you should be allowed to retrieve data
            response = getUserTimeline(twitterUser.id_str)
            # all analysed user will be stored to the same dataframe
            # and its tweets will be agragated into a dataframe within all users tweets retrived.
            dfTimelines = createDataFrameFromTimeline(response, dfTimelines) # should add data to the main dataframe
        else: # timeline closed
            response = getUserTimeline(twitterUser.id_str)
#             print(f'{count} Timeline de :@ {handle} é fechada!')
#             print(f'{response}')
            dfTimelines = createDataFrameFromTimeline("Timeline bloqueada", dfTimelines, error=True, handle=handle) # should add data to the main dataframe
            # throw error.
            # log error in the table
            # move to the next line
#             break    
    else:
#         print(f'{count} O Usuário @{handle} não rolou. Razão: {twitterUser.reason}')
        dfUser = createDataFrameFromUser(twitterUser.reason, dfUser, error=True, handle = handle)
        # move to the next line of the handle dataset
        # log some error into the dataset
        # continue the flux

 ############ 98 # PERFIS PARA ANALISAR
0 # Analisando o perfil:@ weintraub2022sp
1 # Analisando o perfil:@ masbr26
2 # Analisando o perfil:@ betobiagi
3 # Analisando o perfil:@ jikiovi
4 # Analisando o perfil:@ s0ualiada
5 # Analisando o perfil:@ carlosgomes2021
6 # Analisando o perfil:@ karinapaulam
7 # Analisando o perfil:@ ricardoserfert
8 # Analisando o perfil:@ rsdem_crista
9 # Analisando o perfil:@ 142urgente
10 # Analisando o perfil:@ nascime89
11 # Analisando o perfil:@ malupatriota
12 # Analisando o perfil:@ bellazanoniart
13 # Analisando o perfil:@ alvamartaylor
14 # Analisando o perfil:@ fernando_hfa0
15 # Analisando o perfil:@ enzogiovaniss
16 # Analisando o perfil:@ kameen_rider
17 # Analisando o perfil:@ abelbolsoguedes
18 # Analisando o perfil:@ sitelorenzetti
19 # Analisando o perfil:@ jp1982silva
20 # Analisando o perfil:@ helenastatusquo
21 # Analisando o perfil:@ belabonoro
22 # Analisando o perfil:@ criskadutra
23 # Analisando o perfil:@ birolirobrasil
24 # Analisa

In [84]:
dfUser.head()

,error,created_at,default_profile,description,followers_count,friends_count,handle,lang,location,name,profile_image,twitter_id,twitter_is_protected,verified,withheld_in_countries
0,NaN,2017-06-29 20:35:35,1.0,Perfil independente criado para pedir e apoiar...,46018.0,197.0,Weintraub2022SP,None,Brasil,Weintraub 2022 - São Paulo,http://pbs.twimg.com/profile_images/1428085110...,8.805252e+17,0.0,0.0,[]
1,NaN,2021-05-03 03:01:16,True,"Roman catholic, proud right-winger, high schoo...",52.0,150.0,MASBR26,None,República Federativa Do Brasil,Marcelo Amâncio Santos,http://pbs.twimg.com/profile_images/1389052545...,1.389052e+18,False,False,[]
2,NaN,2020-06-23 13:13:20,True,"Art. 5º Todos são iguais perante a lei, sem di...",542.0,1003.0,BetoBiagi,None,,VERDADE ACIMA DE TUDO .,http://pbs.twimg.com/profile_images/1439381602...,1.275417e+18,False,False,[]
3,NaN,2010-07-29 01:53:44,False,“Precisamos perceber a situação. Esta é uma gu...,1396.0,393.0,jikiovi,None,Brasil,Daniel,http://pbs.twimg.com/profile_images/1442545279...,1.721654e+08,False,False,[]
4,NaN,2021-07-11 07:38:53,True,“A vida só presta quando se é feliz offline pr...,1704.0,2050.0,S0uAliada,None,🇧🇷🏖☀️👙,Aliada 🇧🇷 ▄︻┻┳═一🔥,http://pbs.twimg.com/profile_images/1437258349...,1.414127e+18,False,False,[]


In [85]:
dfTimelines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7080 entries, 0 to 7079
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   error                 0 non-null      object        
 1   tweet_author          7080 non-null   object        
 2   tweet_author_id_str   7080 non-null   object        
 3   tweet_contributors    0 non-null      object        
 4   tweet_created_at      7080 non-null   datetime64[ns]
 5   tweet_favorite_count  7080 non-null   float64       
 6   tweet_favorited       7080 non-null   object        
 7   tweet_geo             0 non-null      object        
 8   tweet_hashtags        7080 non-null   object        
 9   tweet_id              7080 non-null   float64       
 10  tweet_id_str          7080 non-null   object        
 11  tweet_is_retweet      7080 non-null   object        
 12  tweet_lang            7080 non-null   object        
 13  tweet_place       

In [86]:
dfUser.to_csv("/Users/dc/Downloads/df_users.csv", sep=',', encoding="utf-8")

In [87]:
dfTimelines.to_csv("/Users/dc/Downloads/df_timelines.csv", sep=',', encoding="utf-8")

In [570]:
dfTimelines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   error                 14 non-null     object        
 1   handle                14 non-null     object        
 2   tweet_author          116 non-null    object        
 3   tweet_author_id_str   116 non-null    object        
 4   tweet_contributors    0 non-null      object        
 5   tweet_created_at      116 non-null    datetime64[ns]
 6   tweet_favorite_count  116 non-null    float64       
 7   tweet_favorited       116 non-null    object        
 8   tweet_geo             0 non-null      object        
 9   tweet_hashtags        116 non-null    object        
 10  tweet_id              116 non-null    float64       
 11  tweet_id_str          116 non-null    object        
 12  tweet_is_retweet      116 non-null    object        
 13  tweet_lang          